<a href="https://colab.research.google.com/github/TranPhu1999/Wordle_solver/blob/main/wordle_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reference:
- 3blue1brown video: https://www.youtube.com/watch?v=v68zYyaEmEA&t=848s
- 3blue1brown repo: https://github.com/3b1b/videos/tree/870a6cbf30938793f93a2c9235c82bdeed31c7c6/_2022/wordle

# Install

In [1]:
!pip install -r /content/drive/MyDrive/Projects/Wordle_solver/requirements.txt

# Dev_v1

In [63]:
# import
import requests
from tqdm import tqdm as ProgressDisplay
from itertools import product
from math import log
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from english_words import get_english_words_set
import numpy as np

# Disable InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [64]:
# Global variable
web2lowerset = get_english_words_set(['web2'], lower=True, alpha =False) # gcide / web2
web2lowerset = list(web2lowerset)
common_letter_rank = "etaonrishdlfcmugypwbvkjxzq"


CORRECT, PRESENT, ABSENT = 2, 1, 0

In [65]:
def check_answer(guess_word: str, seed: int, text_size: int) -> list:
  # Check answer from API and return next pattern
  query = f"https://wordle.votee.dev:8000/random?guess={guess_word}&seed={seed}&size={text_size}"
  r = requests.get(query, verify = False)
  result = [check["result"] for check in r.json()]
  return result

In [66]:
# check_answer("thefo",1234,5)

In [67]:
def get_the_remain_words_list(guess_word: str, pattern: list, allowed_words_list: list) -> list:
  remain_list = []

  for word in allowed_words_list:
    check = True
    for i, possible in enumerate(pattern): # "absent": 0, "present": 1, "correct": 2,
      if (possible == ABSENT and guess_word[i] in word) or \
      (possible == PRESENT and guess_word[i] not in word) or \
      (possible == CORRECT and guess_word[i] != word[i]):
        check = False
    if check:
      remain_list.append(word)
  return remain_list

In [68]:
# get_the_remain_words_list("theao",[2,2,2,0,0],[x for x in web2lowerset  if len(x) == 5])

In [69]:
def calculate_entropy(word, all_pattern: list, allowed_words_list: list)-> float:
  # Return entropy of the correspond word, check the details explanation here: https://youtu.be/v68zYyaEmEA
  # For a particular word, check for all possible pattern and their probability
  # Then calculate the entropy for each of these word with formular of information theory E = sum(px*log(1/px))

  entropy = 0
  for pattern in all_pattern:
    word_basket = get_the_remain_words_list(word, pattern, allowed_words_list)
    if len(word_basket) > 0:
      px = len(word_basket)/len(allowed_words_list)
      entropy += px * log(1/px,2)
  return entropy

In [70]:
def get_best_guess(allowed_words_list: list, all_pattern: list)-> list:
  # from the allow word list, get the word that give the highest entropy
  # higher entropy mean it wil help split the possibility word even smaller
  highest_entropy = 0
  result_index = 0
  for i, word in enumerate(ProgressDisplay(allowed_words_list)):
    entropy = calculate_entropy(word, all_pattern, allowed_words_list)
    if entropy > highest_entropy:
      highest_entropy = entropy
      result_index = i

  return result_index

In [71]:
def recursive_guess(inital_guess: str, allowed_words_list: list, all_pattern: list, seed: int) -> str:
  # recursive function, shrink the allowed_words_list down untill there is one word left or the guess was correct
  if len(allowed_words_list) == 1:
    return allowed_words_list[0]

  guess_pattern = check_answer(inital_guess, seed, len(inital_guess))
  print(f"\ninitial guess: {inital_guess}")
  print(f"guess_pattern: {guess_pattern}")
  if set(guess_pattern) == set(["correct"]):
    return inital_guess
  else:
    #if not true, filter the allowed_words_list with the guess_pattern above
    pattern = {"absent": ABSENT, "present": PRESENT, "correct": CORRECT}
    guess_pattern = [pattern[x] for x in guess_pattern]
    remain_words = get_the_remain_words_list(inital_guess, guess_pattern, allowed_words_list)
    best_guess_index = get_best_guess(remain_words, all_pattern)
    best_guess = remain_words.pop(best_guess_index)
    return recursive_guess(best_guess, remain_words, all_pattern, seed)

In [72]:
def wordle_solver(seed: int, text_size: int) -> str:
  all_pattern = np.array(list(product(*[[ABSENT ,PRESENT ,CORRECT]]*text_size)),dtype=int) # "absent": 0, "present": 1, "correct": 2,
  # main function solve wordle

  allowed_text = []

  for text in web2lowerset:
    if len(text) == text_size and text not in allowed_text:
      allowed_text.append(text)

  answer = recursive_guess(inital_guess = common_letter_rank[:text_size],\
                           allowed_words_list = allowed_text, all_pattern=all_pattern, seed = seed)

  return answer

In [73]:
%%time
wordle_solver(1234, 5)


initial guess: etaon
guess_pattern: ['present', 'present', 'absent', 'absent', 'absent']


100%|██████████| 405/405 [01:14<00:00,  5.42it/s]



initial guess: slite
guess_pattern: ['absent', 'absent', 'absent', 'present', 'present']


100%|██████████| 135/135 [00:07<00:00, 18.38it/s]



initial guess: berth
guess_pattern: ['absent', 'present', 'absent', 'present', 'present']


100%|██████████| 26/26 [00:00<00:00, 91.88it/s]



initial guess: wheft
guess_pattern: ['absent', 'correct', 'correct', 'correct', 'correct']


100%|██████████| 1/1 [00:00<00:00, 1007.28it/s]



initial guess: theft
guess_pattern: ['correct', 'correct', 'correct', 'correct', 'correct']
CPU times: user 1min 21s, sys: 385 ms, total: 1min 21s
Wall time: 1min 28s


'theft'

# Dev_v2

In [1]:
# import
import requests
from tqdm import tqdm as ProgressDisplay
from itertools import product
from math import log
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from english_words import get_english_words_set
import numpy as np

# Disable InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
# Global variable
web2lowerset = get_english_words_set(['web2'], lower=True, alpha =False) # gcide / web2
web2lowerset = list(web2lowerset)
common_letter_rank = "etaonrishdlfcmugypwbvkjxzq"

CORRECT, PRESENT, ABSENT = np.uint8(2), np.uint8(1), np.uint8(0)

In [3]:
def words_to_int_arrays(words):
    return np.array([[ord(c)for c in w] for w in words], dtype=np.uint8)

In [4]:
# https://github.com/3b1b/videos/blob/870a6cbf30938793f93a2c9235c82bdeed31c7c6/_2022/wordle/simulations.py#L104
def create_pattern_matrix(allowed_words_list_1, allowed_words_list_2):
  """
  A pattern for two words represents the wordle-similarity
  pattern (grey -> 0, yellow -> 1, green -> 2) but as an integer
  between 0 and 3^5. Reading this integer in ternary gives the
  associated pattern.

  This function computes the pairwise patterns between two lists
  of words, returning the result as a grid of hash values. Since
  this can be time-consuming, many operations that can be are vectorized
  (perhaps at the expense of easier readibility), and the the result
  is saved to file so that this only needs to be evaluated once, and
  all remaining pattern matching is a lookup.
  """
  text_size = len(allowed_words_list_1[0])
  matrix_dimension_1 = len(allowed_words_list_1)
  matrix_dimension_2 = len(allowed_words_list_2)

  allowed_words_arr_1, allowed_words_arr_2 = map(words_to_int_arrays,(allowed_words_list_1, allowed_words_list_2))

  # equality_grid keeps track of all equalities between all pairs
  # of letters in words. Specifically, equality_grid[a, b, i, j]
  # is true when words[i][a] == words[b][j]
  equality_grid = np.zeros((matrix_dimension_1, matrix_dimension_2, text_size, text_size), dtype=bool)
  for i, j in product(range(text_size), range(text_size)):
      equality_grid[:, :, i, j] = np.equal.outer(allowed_words_arr_1[:, i], allowed_words_arr_2[:, j])

  # full_pattern_matrix[a, b] should represent the 5-color pattern
  # for guess a and answer b, with 0 -> grey, 1 -> yellow, 2 -> green
  full_pattern_matrix = np.zeros((matrix_dimension_1, matrix_dimension_2, text_size), dtype=np.uint8)

  # Green pass
  for i in range(text_size):
      matches = equality_grid[:, :, i, i].flatten()  # matches[a, b] is true when words[a][i] = words[b][i]
      full_pattern_matrix[:, :, i].flat[matches] = CORRECT

      for k in range(text_size):
          # If it's a match, mark all elements associated with
          # that letter, both from the guess and answer, as covered.
          # That way, it won't trigger the yellow pass.
          equality_grid[:, :, k, i].flat[matches] = False
          equality_grid[:, :, i, k].flat[matches] = False

  # Yellow pass
  for i, j in product(range(text_size), range(text_size)):
      matches = equality_grid[:, :, i, j].flatten()
      full_pattern_matrix[:, :, i].flat[matches] = PRESENT
      for k in range(text_size):
          # Similar to above, we want to mark this letter
          # as taken care of, both for answer and guess
          equality_grid[:, :, k, j].flat[matches] = False
          equality_grid[:, :, i, k].flat[matches] = False

  # Rather than representing a color pattern as a lists of integers,
  # store it as a single integer, whose ternary representations corresponds
  # to that list of integers.
  pattern_matrix = np.dot(
      full_pattern_matrix,
      (3**np.arange(text_size)).astype(np.uint8)
  )

  return pattern_matrix

In [5]:
def generate_full_pattern_matrix(save_file_name, allowed_text):
    pattern_matrix = create_pattern_matrix(allowed_text, allowed_text)
    # Save to file
    np.save(save_file_name, pattern_matrix)
    return pattern_matrix

In [6]:
text_basket_with_size = dict()
for text in web2lowerset:
  if len(text) in text_basket_with_size:
    text_basket_with_size[len(text)].append(text)
  else:
    text_basket_with_size[len(text)] = [text]

In [ ]:
generate_full_pattern_matrix(f"pattern_matrix_size_{2}.npy",text_basket_with_size[5])

In [ ]:
for length, list_word in ProgressDisplay(text_basket_with_size.items()):
  import os
  pattern_file = f"pattern_matrix_size_{length}.npy"
  if not os.path.exists(pattern_file):
    generate_full_pattern_matrix(pattern_file ,list_word)

  0%|          | 0/25 [00:00<?, ?it/s]

In [78]:
def check_answer(guess_word: str, seed: int, text_size: int) -> list:
  # Check answer from API and return next pattern
  query = f"https://wordle.votee.dev:8000/random?guess={guess_word}&seed={seed}&size={text_size}"
  r = requests.get(query, verify = False)
  result = [check["result"] for check in r.json()]
  return result

In [79]:
# check_answer("thefo",1234,5)

In [80]:
def get_the_remain_words_list(guess_word: str, pattern: list, allowed_words_list: list) -> list:
  remain_list = []

  for word in allowed_words_list:
    check = True
    for i, possible in enumerate(pattern): # "absent": 0, "present": 1, "correct": 2,
      if (possible == ABSENT and guess_word[i] in word) or \
      (possible == PRESENT and guess_word[i] not in word) or \
      (possible == CORRECT and guess_word[i] != word[i]):
        check = False
    if check:
      remain_list.append(word)
  return remain_list

In [81]:
# get_the_remain_words_list("theao",[2,2,2,0,0],[x for x in web2lowerset  if len(x) == 5])

In [82]:
def calculate_entropy(word, all_pattern: list, allowed_words_list: list)-> float:
  # Return entropy of the correspond word, check the details explanation here: https://youtu.be/v68zYyaEmEA
  # For a particular word, check for all possible pattern and their probability
  # Then calculate the entropy for each of these word with formular of information theory E = sum(px*log(1/px))

  entropy = 0
  for pattern in all_pattern:
    word_basket = get_the_remain_words_list(word, pattern, allowed_words_list)
    if len(word_basket) > 0:
      px = len(word_basket)/len(allowed_words_list)
      entropy += px * log(1/px,2)
  return entropy

In [83]:
def get_best_guess(allowed_words_list: list, all_pattern: list)-> list:
  # from the allow word list, get the word that give the highest entropy
  # higher entropy mean it wil help split the possibility word even smaller
  highest_entropy = 0
  result_index = 0
  for i, word in enumerate(ProgressDisplay(allowed_words_list)):
    entropy = calculate_entropy(word, all_pattern, allowed_words_list)
    if entropy > highest_entropy:
      highest_entropy = entropy
      result_index = i

  return result_index

In [84]:
def recursive_guess(inital_guess: str, allowed_words_list: list, all_pattern: list, seed: int) -> str:
  # recursive function, shrink the allowed_words_list down untill there is one word left or the guess was correct
  if len(allowed_words_list) == 1:
    return allowed_words_list[0]

  guess_pattern = check_answer(inital_guess, seed, len(inital_guess))
  print(f"\ninitial guess: {inital_guess}")
  print(f"guess_pattern: {guess_pattern}")
  if set(guess_pattern) == set(["correct"]):
    return inital_guess
  else:
    #if not true, filter the allowed_words_list with the guess_pattern above
    pattern = {"absent": ABSENT, "present": PRESENT, "correct": CORRECT}
    guess_pattern = [pattern[x] for x in guess_pattern]
    remain_words = get_the_remain_words_list(inital_guess, guess_pattern, allowed_words_list)
    best_guess_index = get_best_guess(remain_words, all_pattern)
    best_guess = remain_words.pop(best_guess_index)
    return recursive_guess(best_guess, remain_words, all_pattern, seed)

In [87]:
def wordle_solver(seed: int, text_size: int) -> str:
  all_pattern = list(product(*[[ABSENT ,PRESENT ,CORRECT]]*text_size)) # "absent": 0, "present": 1, "correct": 2,
  # main function solve wordle

  allowed_text = []

  for text in web2lowerset:
    if len(text) == text_size and text not in allowed_text:
      allowed_text.append(text)

  answer = recursive_guess(inital_guess = common_letter_rank[:text_size],\
                           allowed_words_list = allowed_text, all_pattern=all_pattern, seed = seed)

  return answer

In [88]:
%%time
wordle_solver(1234, 5)


initial guess: etaon
guess_pattern: ['present', 'present', 'absent', 'absent', 'absent']


100%|██████████| 405/405 [00:26<00:00, 15.01it/s]



initial guess: slite
guess_pattern: ['absent', 'absent', 'absent', 'present', 'present']


100%|██████████| 135/135 [00:02<00:00, 50.01it/s]



initial guess: berth
guess_pattern: ['absent', 'present', 'absent', 'present', 'present']


100%|██████████| 26/26 [00:00<00:00, 253.52it/s]



initial guess: wheft
guess_pattern: ['absent', 'correct', 'correct', 'correct', 'correct']


100%|██████████| 1/1 [00:00<00:00, 2239.35it/s]



initial guess: theft
guess_pattern: ['correct', 'correct', 'correct', 'correct', 'correct']
CPU times: user 30.2 s, sys: 208 ms, total: 30.4 s
Wall time: 35.6 s


'theft'